In [112]:
from typing import Union
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

%time
config = dict(
    tokenizer=AutoTokenizer.from_pretrained("unitary/toxic-bert"),
    model=AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert"),
)

Wall time: 0 ns


In [142]:
# inference function
def hate_speech_inference(query:Union[str, list], tokenizer, model) -> dict:
    id2label = model.config.id2label
    # disable pytorch automatic differentiation engine
    with torch.no_grad():
        tokens = tokenizer(query, return_tensors='pt', padding=True, truncation=True) # convert text to token i.e. word to numbers
        logits = model(**tokens).logits # run inference
        proba = torch.sigmoid(logits) # sigmoid probabilities
        pred_class = proba.apply_(lambda x: 1 if x >= 0.5 else 0) # predict class as one-hot encodings
        return dict(
            proba = proba,
            id2label = id2label,
            pred_class = pred_class
        )

In [143]:
# single instance prediction
hate_speech_inference('COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK', **config) # multiple toxic traits

{'proba': tensor([[1., 0., 1., 0., 1., 0.]]),
 'id2label': {0: 'toxic',
  1: 'severe_toxic',
  2: 'obscene',
  3: 'threat',
  4: 'insult',
  5: 'identity_hate'},
 'pred_class': tensor([[1., 0., 1., 0., 1., 0.]])}

In [147]:
input_batch = [
    "COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK", # toxic, severe toxic, obscene, insult
    "Oh, and the girl above started her arguments with me. She stuck her nose where it doesn't belong. I believe the argument was between me and Yvesnimmo. But like I said, the situation was settled and I apologized. Thanks,", # non-toxic
    "Bye!", # might be toxic
    "You are gay or antisemmitian? " # toxic and identity hate
]

hate_speech_inference(input_batch, **config)

{'proba': tensor([[1., 0., 1., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 1.]]),
 'id2label': {0: 'toxic',
  1: 'severe_toxic',
  2: 'obscene',
  3: 'threat',
  4: 'insult',
  5: 'identity_hate'},
 'pred_class': tensor([[1., 0., 1., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 1.]])}

## Model Architecture

In [ ]:
import torchinfo
torchinfo.summary(config['model'])

take note: I'm looking to do batch inferencing next